# Bloque 4 · Integrando RAG con Chainlit

En los bloques anteriores construimos la infraestructura de RAG: comprendimos el flujo, simulamos la búsqueda vectorial, aprendimos a invocar la knowledge base en Bedrock y utilizamos la API de RAG para generar respuestas con LLM. Este bloque muestra cómo presentar todo ese trabajo mediante un chatbot sencillo creado con Chainlit.

## ¿Por qué utilizar Chainlit?

- **Interfaz lista para usar**: en minutos tenemos un chat web funcional sin construir un front-end desde cero.
- **Orientado a flujos conversacionales**: facilita manejar el estado de la conversación, mostrar componentes ricos (citas, tablas) y registrar interacciones.
- **Productividad**: nos permite iterar sobre la experiencia del usuario mientras seguimos afinando el backend RAG.
- **Integración sencilla**: basta con una función `on_chat_start` para inicializar recursos y un handler `on_message` que orqueste la recuperación y generación.

En nuestra arquitectura, Chainlit ocupa la capa de presentación: recibe la pregunta del usuario, llama a nuestras funciones en Python (que a su vez invocan Bedrock) y muestra la respuesta final con las citas correspondientes.

## Flujo general

1. El usuario escribe un mensaje en el chatbot.
2. Chainlit llama a nuestra lógica Python.
3. Esa lógica reutiliza las funciones del Bloque 3 (`generar_con_prompt`, `mostrar_generacion`, etc.).
4. Bedrock recupera fragmentos, genera la respuesta y nos devuelve citas.
5. Chainlit muestra el resultado en la interfaz, dando contexto y confianza al usuario.

La actividad consiste en ejecutar un chatbot prearmado, modificar parámetros sencillos y verificar que las respuestas coinciden con lo que obtuvimos en el Bloque 3.

## Preparación del entorno

1. Ejecuta la celda de instalación del **Bloque 0** (ya incluye las dependencias comunes).
2. Instala Chainlit desde esta notebook si aún no está disponible:

```python
!pip install chainlit
```

3. Asegúrate de que las variables `BEDROCK_KB_ID`, `BEDROCK_MODEL_ARN`, `BEDROCK_DATA_SOURCE_ID` y `AWS_REGION` permanecen configuradas (igual que en los bloques 2 y 3).
4. Confirma que las funciones del Bloque 3 están disponibles en el entorno (puedes copiar las definiciones relevantes a un módulo común o importarlas desde el propio notebook).

## Estructura básica del proyecto Chainlit

Chainlit busca un archivo principal (por defecto `app.py`) con dos funciones:

- `@cl.on_chat_start`: se ejecuta cuando el usuario abre la sesión. Aquí podemos cargar prompts, inicializar clientes y mostrar un mensaje de bienvenida.
- `@cl.on_message`: se ejecuta por cada mensaje del usuario. Debe invocar nuestra lógica RAG y enviar la respuesta al chat.

Crearemos `app.py` en el directorio raíz para que puedas ejecutarlo con `chainlit run app.py -w` (el flag `-w` habilita recarga automática).

### Archivo `app.py`

Puedes copiar el siguiente ejemplo en tu proyecto (no se editan archivos desde la notebook para evitar conflictos, pero si lo prefieres crea el fichero manualmente).

```python
import os
import chainlit as cl

from rag_runtime import generar_con_prompt, DEFAULT_PROMPT_TEMPLATE
from rag_runtime import mostrar_generacion_simple


PROMPT_TEMPLATE = os.getenv("CHAINLIT_PROMPT_TEMPLATE", DEFAULT_PROMPT_TEMPLATE)


@cl.on_chat_start
async def on_chat_start():
    await cl.Message(
        content=(
            "Hola 👋 Soy el asistente interno. Puedes hacer preguntas sobre la documentación"
            " corporativa y citaré las fuentes si están disponibles."
        )
    ).send()


@cl.on_message
async def on_message(message: cl.Message):
    pregunta = message.content
    await cl.Message(content="Procesando tu pregunta...").send()

    respuesta = generar_con_prompt(pregunta, PROMPT_TEMPLATE)
    texto, citas = mostrar_generacion_simple(respuesta)

    await cl.Message(content=texto, author="Asistente RAG").send()

    if citas:
        citas_md = "\n".join(f"- {cita}" for cita in citas)
        await cl.Message(content=f"Fuentes:\n{citas_md}").send()
```

Notas:
- `rag_runtime` representa un módulo reutilizable donde puedes colocar las funciones del Bloque 3 (`generar_con_prompt` y una variante de `mostrar_generacion` que devuelva texto y citas).
- Para mantener el notebook conciso, solo mostramos un helper `mostrar_generacion_simple` que transforma la respuesta en texto y una lista de citas (puedes implementarlo igual que en el Bloque 3).
- `CHAINLIT_PROMPT_TEMPLATE` permite experimentar con prompts sin editar el código.

## Actividad · Ejecutar el chatbot y validar la integración

1. Copia `app.py` en la raíz del proyecto junto con el módulo `rag_runtime.py` (que puede contener las funciones del Bloque 3).
2. En una terminal, ejecuta:
   ```bash
   chainlit run app.py -w
   ```
3. Abre el enlace que Chainlit mostrará en la consola (usualmente `http://localhost:8000`).
4. Haz preguntas similares a las de los bloques anteriores y comprueba que las respuestas coinciden con lo esperado y que se muestran las citas.
5. Modifica la variable `CHAINLIT_PROMPT_TEMPLATE` o las funciones de `rag_runtime` para personalizar el comportamiento.

La actividad no requiere construir el chatbot desde cero; el objetivo es entender cómo conectar los componentes y verificar que funcionan de punta a punta.

## Extensiones sugeridas

- Mostrar el top de fragmentos recuperados en tarjetas Chainlit antes de la respuesta final.
- Agregar botones de seguimiento (por ejemplo, "Agregar documento" o "Reintentar con otro prompt").
- Integrar métricas de uso u observabilidad para monitorear las conversaciones.
- Configurar autenticación para restringir el acceso al chatbot.

## Rol de Chainlit en la arquitectura

Resumiendo:
- **Recuperación y generación** (Bloques 2 y 3) siguen viviendo en Python y Bedrock.
- **Chainlit** actúa como la interfaz conversacional que conecta a nuestros usuarios con el flujo RAG.
- Este patrón desacopla la lógica del chatbot del front-end, lo que facilita experimentar con prompts, cambiar modelos o añadir nuevas fuentes sin reescribir la interfaz.

## Próximos pasos

- Consolidar las funciones de recuperación y generación en un paquete o módulo reutilizable.
- Añadir controles adicionales (por ejemplo, manejo de errores más detallado o logging).
- Preparar un entorno de demostración/producción en la nube para que el equipo pueda probar el chatbot sin depender de notebooks.
- Evaluar la necesidad de moderación de contenidos y políticas de seguridad antes de liberar el asistente.